In [ ]:
api_key =   #replace it with openAI api key


# Provide the folder ID from the Google Drive link where item embeddings are loaded
folder_id = '18Ryk87NyHuGydGvhNYgBG3d0bpbBX8r_'
load_trained_data = 1
run_openAI_experiments = 1-load_trained_data

In [ ]:
!pip install openai==0.28.0
!pip install faiss-cpu
# Install PyDrive
!pip install -U -q PyDrive
import openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 20.1 MB/s eta 0:00:00


## Utility Functions

In [ ]:
import numpy as np
import pandas as pd
def print_statistics(data):
    total_interactions = data.shape[0]
    unique_users = data['userId'].nunique()
    unique_items = data['itemId'].nunique()

    print("--------------------------")
    print("The Dataset Overview:")
    print(f" - Total Interactions: {total_interactions}")
    print(f" - |U|: {unique_users}")
    print(f" - |I|: {unique_items}")
    print(f" - |R|/|U|: {total_interactions / unique_users:.2f}")
    print(f" - |R|/|I|: {total_interactions / unique_items:.2f}")
    print(f" - |R|/(|U|*|I|): {total_interactions / (unique_users * unique_items):.10f}")
    print("--------------------------")
# print_statistics(train_df)
# print_statistics(test_df)


def sample_users(df_ratings, num_users):
    # Get a list of unique users
    unique_users = df_ratings['userId'].unique()

    # Sample a specified number of users
    sampled_users = pd.Series(unique_users).sample(n=num_users, random_state=42)

    # Get all ratings for the sampled users
    df_sampled_ratings = df_ratings[df_ratings['userId'].isin(sampled_users)]

    return df_sampled_ratings


def apply_interaction_limits(data, min_interactions, max_interactions, user_col, item_col):
    # Count the number of interactions by user
    interaction_counts = data[user_col].value_counts()

    # Filter out users outside the interaction limits
    users_to_keep = interaction_counts[(interaction_counts >= min_interactions) & (interaction_counts <= max_interactions)].index
    return data[data[user_col].isin(users_to_keep)]


def analyze_tags(df_item):
    # Count unique item IDs
    unique_item_ids = df_item['itemId'].nunique()

    # Count unique tags across all items (flattening the lists first)
    unique_tags = set(tag for sublist in df_item['tag'] for tag in sublist)
    unique_tags_count = len(unique_tags)

    # Count how many items have at least one tag
    items_with_tags = df_item[df_item['tag'].apply(lambda tags: len(tags) > 0)].shape[0]

    # Count how many items have no tags
    total_items = df_item.shape[0]
    items_without_tags = total_items - items_with_tags

    # Normalize to the entire number of items to get percentages
    items_with_tags_percentage = (items_with_tags / total_items) * 100
    items_without_tags_percentage = (items_without_tags / total_items) * 100

    # Print the results
    print(f"Total unique items (itemId): {unique_item_ids}")
    print(f"Total unique tags: {unique_tags_count}")
    print(f"Items with at least one tag: {items_with_tags} ({items_with_tags_percentage:.2f}%)")
    print(f"Items with no tags: {items_without_tags} ({items_without_tags_percentage:.2f}%)")


def check_column_for_nan(df, column_name):
    """
    Checks if a column in a DataFrame, identified by its name, contains NaN values.

    Parameters:
    df (pd.DataFrame): The DataFrame to check.
    column_name (str): The name of the column to check for NaN values.

    Returns:
    int: 1 if NaN values are found, 0 if no NaN values are present.
    """
    # Ensure the column_name exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame.")

    # Return 1 if NaN values are found, otherwise return 0
    return int(df[column_name].isna().any())


# Introduce a NaN value in the 'tags' column
data = {
    'title': ['Movie A', 'Movie B', 'Movie C'],
    'genres': ['Action', 'Drama', 'Comedy'],
    'tags': ['Tag1', np.nan, 'Tag3']  # Introducing NaN in the 'tags' column
}

df = pd.DataFrame(data)
display(df)
# Check if the 'tags' column has NaN values
result = check_column_for_nan(df, 'tags')
print(f"'tags' column has NaN values: {result}")

# Check if the 'title' column has NaN values
result = check_column_for_nan(df, 'title')
print(f"'title' column has NaN values: {result}")


,title,genres,tags
0,Movie A,Action,Tag1
1,Movie B,Drama,NaN
2,Movie C,Comedy,Tag3


'tags' column has NaN values: 1
'title' column has NaN values: 0


In [ ]:
import requests
import pandas as pd

def download_csv(url, csv_file_name):
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        with open(csv_file_name, "wb") as file:
            for chunk in response.iter_content(chunk_size=128):
                file.write(chunk)
        print(f"{csv_file_name} downloaded successfully.")
        return True
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
        return False

def load_datasets():
    # URLs for the ratings, items, and users datasets
    urlRatings = "https://raw.githubusercontent.com/yasdel/RecSys_Datasets/e3a18487525d3ae8ec19b39959c1a881c383e1a8/ml_latest/df_ratings_ml-latest-small.csv"
    urlItems = "https://raw.githubusercontent.com/yasdel/RecSys_Datasets/e3a18487525d3ae8ec19b39959c1a881c383e1a8/ml_latest/df_item_ml-latest-small.csv"
    urlTags = "https://raw.githubusercontent.com/yasdel/RecSys_Datasets/21484da12f01263fc0194d12c63d796628f84137/ml_latest/df_tags_ml-latest-small.csv"

    # Local filenames
    ratings_file = "df_ratings_ml-latest-small.csv"
    items_file = "df_item_ml-latest-small.csv"
    tags_file = "df_tags_ml-latest-small.csv"

    # Download the ratings, items, and users files
    if download_csv(urlRatings, ratings_file) and download_csv(urlItems, items_file) and download_csv(urlTags, tags_file):
        column_name = ['userId', 'itemId', 'rating', 'timestamp']
        df_ratings = pd.read_csv(ratings_file, sep=',', low_memory=False, header= 0 , names=column_name)
        df_items = pd.read_csv(items_file, sep=',', low_memory=False)
        df_tags = pd.read_csv(tags_file, sep=',', low_memory=False)
        df_ratings['userId'] = df_ratings['userId'].astype(int)
        df_ratings['itemId'] = df_ratings['itemId'].astype(int)
        df_items['itemId'] = df_items['itemId'].astype(int)
        df_tags['itemId'] = df_tags['itemId'].astype(int)
        df_tags['userId'] = df_tags['userId'].astype(int)

        # Group the tags by itemId and aggregate them into a list
        tags_grouped = df_tags.groupby('itemId')['tag'].apply(list).reset_index()

        # Merge the tags with the df_item to create a new column 'tags'
        df_items_merged = pd.merge(df_items, tags_grouped, on='itemId', how='left')

        # If there are items without tags, they will have NaN in the 'tags' column; replace with an empty list
        df_items_merged['tag'] = df_items_merged['tag'].apply(lambda x: x if isinstance(x, list) else [])

        print("ml-latest-small Datasets and tags loaded successfully.")
        print("Datasets loaded successfully.")
        return df_ratings, df_items,  df_tags, df_items_merged
    else:
        print("Failed to download one or more datasets.")
        return None, None, None

DS_name = "ml-latest-small"
# Usage example for ml-latest-small ratings, items, and users datasets
df_ratings, df_items, df_tags, df_items_merged = load_datasets()

print_statistics(df_ratings)
display(df_ratings.head(10))

df_ratings_ml-latest-small.csv downloaded successfully.
df_item_ml-latest-small.csv downloaded successfully.
df_tags_ml-latest-small.csv downloaded successfully.
ml-latest-small Datasets and tags loaded successfully.
Datasets loaded successfully.
--------------------------
The Dataset Overview:
 - Total Interactions: 100836
 - |U|: 610
 - |I|: 9724
 - |R|/|U|: 165.30
 - |R|/|I|: 10.37
 - |R|/(|U|*|I|): 0.0169996831
--------------------------


,userId,itemId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [ ]:
df_items

,itemId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


##Load and Read Pretrained Item Embeddings

In [ ]:
# Import necessary libraries
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List of specific .csv file names to download
specific_files = [
    "train_data_enriched_dataset_ml-latest-small_filtering_min_20_max_100.csv",
    "test_data_enriched_dataset_ml-latest-small_filtering_min_20_max_100.csv",
    "df_items_withPopularity.csv"
]

# Function to list and download specific files from a Google Drive folder
def download_files_from_folder(folder_id):
    # List all files in the folder
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

    print(f'Found {len(file_list)} files in the folder.')

    # Create a local directory to save downloaded files
    if not os.path.exists('drive_folder'):
        os.makedirs('drive_folder')

    # Download each specified file
    for file in file_list:
        if file['title'] in specific_files:  # Check if file is in the list
            print(f'Downloading {file["title"]}...')
            file.GetContentFile(f'drive_folder/{file["title"]}')
            print(f'Downloaded: {file["title"]}')
        else:
            print(f'Skipping {file["title"]}...')

# Call the function to download files
download_files_from_folder(folder_id)

# List the downloaded files
print("Downloaded files:")
!ls drive_folder

fileName = "drive_folder/" + specific_files[0]
train_df = pd.read_csv(fileName)

fileName = "drive_folder/" + specific_files[1]
test_df = pd.read_csv(fileName)

fileName = "drive_folder/" + specific_files[2]
df_items_with_populairty = pd.read_csv(fileName)

display(train_df)
display(test_df)
display(df_items_with_populairty)

# Combine train_df and test_df to create rating_df
rating_df = pd.concat([train_df, test_df], ignore_index=True)

# Display the resulting rating_df
print(rating_df)

Found 32 files in the folder.
Skipping df_items_adversarialAttack_with_embeddings_tag_local_7.csv...
Skipping df_items_adversarialAttack_with_embeddings_tag_local_5.csv...
Skipping df_items_adversarialAttack_with_embeddings_generated_tag_local_7.csv...
Skipping df_items_adversarialAttack_with_embeddings_generated_tag_global_7.csv...
Skipping df_items_adversarialAttack_with_embeddings_tag_global_7.csv...
Skipping df_items_adversarialAttack_with_embeddings_tag_global_5.csv...
Downloaded: df_items_withPopularity.csv
Skipping df_items_adversarialAttack_with_embeddings_tag_7.csv...
Skipping df_items_adversarialAttack_with_embeddings_tag_5.csv...
Skipping df_items_adversarialAttack_with_embeddings_generated_tag_local_5.csv...
Skipping df_items_adversarialAttack_with_embeddings_generated_tag_global_5.csv...
Skipping df_items_adversarialAttack_with_embeddings_generated_tags_7.csv...
Skipping df_items_adversarialAttack_with_embeddings_generated_tags_5.csv...
Skipping df_items_adversarialAttack_

,userId,itemId,rating,timestamp
0,1,1210,5.0,2000-07-30 18:08:19
1,1,2018,5.0,2000-07-30 18:08:43
2,1,2628,4.0,2000-07-30 18:08:43
3,1,2826,4.0,2000-07-30 18:08:43
4,1,3578,5.0,2000-07-30 18:11:08
...,...,...,...,...
65138,610,5959,3.5,2017-05-03 21:39:38
65139,610,60766,4.5,2017-05-03 21:39:43
65140,610,112171,3.5,2017-05-03 21:39:49
65141,610,55721,3.5,2017-05-03 21:40:11


,userId,itemId,rating,timestamp
0,1,1219,2.0,2000-07-30 18:56:33
1,1,1348,4.0,2000-07-30 18:56:33
2,1,2644,4.0,2000-07-30 18:56:33
3,1,1258,3.0,2000-07-30 18:56:54
4,1,1278,5.0,2000-07-30 18:56:54
...,...,...,...,...
16590,610,3727,4.0,2017-05-28 08:14:25
16591,610,70,4.0,2017-05-28 08:14:42
16592,610,328,3.5,2017-05-28 08:14:59
16593,610,2459,3.5,2017-05-28 08:16:45


,itemId,title,genres,num_interactions,popularity_level
0,1500,Grosse Pointe Blank (1997),Comedy|Crime|Romance,59,2
1,367,"Mask, The (1994)",Action|Comedy|Crime|Fantasy,157,2
2,84152,Limitless (2011),Sci-Fi|Thriller,30,1
3,3717,Gone in 60 Seconds (2000),Action|Crime,61,2
4,3238,Eye of the Beholder (1999),Thriller,6,1
...,...,...,...,...,...
3074,5404,84 Charing Cross Road (1987),Drama|Romance,3,0
3075,53894,Sicko (2007),Documentary|Drama,14,1
3076,1342,Candyman (1992),Horror|Thriller,11,1
3077,2917,Body Heat (1981),Crime|Thriller,10,1


       userId  itemId  rating            timestamp
0           1    1210     5.0  2000-07-30 18:08:19
1           1    2018     5.0  2000-07-30 18:08:43
2           1    2628     4.0  2000-07-30 18:08:43
3           1    2826     4.0  2000-07-30 18:08:43
4           1    3578     5.0  2000-07-30 18:11:08
...       ...     ...     ...                  ...
81733     610    3727     4.0  2017-05-28 08:14:25
81734     610      70     4.0  2017-05-28 08:14:42
81735     610     328     3.5  2017-05-28 08:14:59
81736     610    2459     3.5  2017-05-28 08:16:45
81737     610    3917     4.0  2017-05-28 08:16:51

[81738 rows x 4 columns]


In [ ]:
import pandas as pd

# Set a fixed seed for reproducibility
seed = 42

# Step 1: Select 200 random users from `test_df`
random_users = test_df['userId'].drop_duplicates().sample(n=200, random_state=seed)

# Step 2: Filter `train_df` and `test_df` to include only the selected users
train_df = train_df[train_df['userId'].isin(random_users)]
test_df = test_df[test_df['userId'].isin(random_users)]

# Step 3: Get unique `itemId`s from both `filtered_train_df` and `filtered_test_df`
available_item_ids = pd.concat([train_df['itemId'], test_df['itemId']]).unique()

# Step 4: Filter `item_df` to include only the available `itemId`s
df_items_with_populairty = df_items_with_populairty[df_items_with_populairty['itemId'].isin(available_item_ids)]

# Display or df_items_with_populairty the resulting DataFrames as needed
display(train_df)
display(test_df)
display(available_item_ids)
display(df_items_with_populairty)


,userId,itemId,rating,timestamp
0,1,1210,5.0,2000-07-30 18:08:19
1,1,2018,5.0,2000-07-30 18:08:43
2,1,2628,4.0,2000-07-30 18:08:43
3,1,2826,4.0,2000-07-30 18:08:43
4,1,3578,5.0,2000-07-30 18:11:08
...,...,...,...,...
65138,610,5959,3.5,2017-05-03 21:39:38
65139,610,60766,4.5,2017-05-03 21:39:43
65140,610,112171,3.5,2017-05-03 21:39:49
65141,610,55721,3.5,2017-05-03 21:40:11


,userId,itemId,rating,timestamp
0,1,1219,2.0,2000-07-30 18:56:33
1,1,1348,4.0,2000-07-30 18:56:33
2,1,2644,4.0,2000-07-30 18:56:33
3,1,1258,3.0,2000-07-30 18:56:54
4,1,1278,5.0,2000-07-30 18:56:54
...,...,...,...,...
16590,610,3727,4.0,2017-05-28 08:14:25
16591,610,70,4.0,2017-05-28 08:14:42
16592,610,328,3.5,2017-05-28 08:14:59
16593,610,2459,3.5,2017-05-28 08:16:45


array([ 1210,  2018,  2628, ..., 58332,  4852,  3616])

,itemId,title,genres,num_interactions,popularity_level
0,1500,Grosse Pointe Blank (1997),Comedy|Crime|Romance,59,2
1,367,"Mask, The (1994)",Action|Comedy|Crime|Fantasy,157,2
2,84152,Limitless (2011),Sci-Fi|Thriller,30,1
3,3717,Gone in 60 Seconds (2000),Action|Crime,61,2
4,3238,Eye of the Beholder (1999),Thriller,6,1
...,...,...,...,...,...
3073,2002,Lethal Weapon 3 (1992),Action|Comedy|Crime|Drama,46,2
3075,53894,Sicko (2007),Documentary|Drama,14,1
3076,1342,Candyman (1992),Horror|Thriller,11,1
3077,2917,Body Heat (1981),Crime|Thriller,10,1


In [ ]:
# Import necessary libraries
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Folder ID for the folder containing the files
folder_id = "1cWShmJjyq8iLA3vmhC8dA7fqhniog-PK"

# Function to list and download all CSV files from a Google Drive folder
def download_csv_files_from_folder(folder_id):
    # List all files in the folder
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

    print(f'Found {len(file_list)} files in the folder.')

    # Create a local directory to save downloaded files
    if not os.path.exists('drive_folder'):
        os.makedirs('drive_folder')

    # Download each CSV file
    for file in file_list:
        if file['title'].endswith('.csv'):  # Check if file is a CSV
            print(f'Downloading {file["title"]}...')
            file.GetContentFile(f'drive_folder/{file["title"]}')
            print(f'Downloaded: {file["title"]}')
        else:
            print(f'Skipping {file["title"]} (not a CSV file)')

# Call the function to download all CSV files
download_csv_files_from_folder(folder_id)

# List the downloaded files
print("Downloaded files:")
!ls drive_folder

Found 41 files in the folder.
Downloaded: candidResults_reranked_recResults_df_items_adversarialAttack_with_embeddings_generated_tag_global_5.csv
Downloaded: candidResults_reranked_recResults_df_items_adversarialAttack_with_embeddings_generated_tag_local_5.csv
Downloaded: candidResults_reranked_recResults_df_items_adversarialAttack_with_embeddings_generated_tag_global_3.csv
Downloaded: candidResults_reranked_recResults_df_items_adversarialAttack_with_embeddings_generated_tag_local_3.csv
Skipping User_embedding_df_items_adversarialAttack_with_embeddings_generated_tag_local_5.csv_lamb0.01_pow_1.2.pkl (not a CSV file)
Skipping User_embedding_df_items_adversarialAttack_with_embeddings_generated_tag_local_3.csv_lamb0.01_pow_1.2.pkl (not a CSV file)
Skipping User_embedding_df_items_adversarialAttack_with_embeddings_generated_tag_local_1.csv_lamb0.01_pow_1.2.pkl (not a CSV file)
Skipping User_embedding_df_items_adversarialAttack_with_embeddings_generated_tag_global_3.csv_lamb0.01_pow_1.2.pkl 

In [ ]:
files = ['Orig_notAttacked', 'GenTag_notAttacked',
         'Orig_Attacked_tag_1_local', 'Orig_Attacked_tag_1_global',
         'Orig_Attacked_tag_3_local', 'Orig_Attacked_tag_3_global',
         'Orig_Attacked_tag_5_local', 'Orig_Attacked_tag_5_global',
         'Orig_Attacked_tag_7_local', 'Orig_Attacked_tag_7_global',
         'GenTag_Attacked_tag_1_local', 'GenTag_Attacked_tag_1_global',
         'GenTag_Attacked_tag_3_local', 'GenTag_Attacked_tag_3_global'
         'GenTag_Attacked_tag_5_local', 'GenTag_Attacked_tag_5_global',
         'GenTag_Attacked_tag_7_local', 'GenTag_Attacked_tag_7_global']


scenario = files[1]

if scenario == files[0]:
  # this contain embedding for movie in the form of [title, genre, tags]
  file = "recResults_enriched_dataset_ml-latest-small_filtering_min_20_max_100_gpt-3.5-turbo_Embedding_TitleGenreTags_text-embedding-ada-002.csv"

elif scenario == files[1]:
  # this contain embedding for movie in the form of [title, genre, gen_tags]
  file = "recResults_enriched_dataset_ml-latest-small_filtering_min_20_max_100_gpt-3.5-turbo_Embedding_TitleGenreGenTags_text-embedding-ada-002.csv"

elif scenario == files[2]:
  # this contain embedding for movie in the form of [title, genre, gen_tags]
  file = "recResults_df_items_adversarialAttack_with_embeddings_tag_local_1.csv"

elif scenario == files[3]:
  # this contain embedding for movie in the form of [title, genre, gen_tags]
  file = "recResults_df_items_adversarialAttack_with_embeddings_tag_global_1.csv"

elif scenario == files[4]:
  # this contain embedding for movie in the form of [title, genre, gen_tags]
  file = "recResults_df_items_adversarialAttack_with_embeddings_tag_local_3.csv"

elif scenario == files[5]:
  # this contain embedding for movie in the form of [title, genre, gen_tags]
  file = "recResults_df_items_adversarialAttack_with_embeddings_tag_global_3.csv"

elif scenario == files[6]:
  # this contain embedding for movie in the form of [title, genre, gen_tags]
  file = "recResults_df_items_adversarialAttack_with_embeddings_tag_local_5.csv"

elif scenario == files[7]:
  # this contain embedding for movie in the form of [title, genre, gen_tags]
  file = "recResults_df_items_adversarialAttack_with_embeddings_tag_global_5.csv"

elif scenario == files[8]:
  # this contain embedding for movie in the form of [title, genre, gen_tags]
  file = "recResults_df_items_adversarialAttack_with_embeddings_tag_local_7.csv"

elif scenario == files[9]:
  # this contain embedding for movie in the form of [title, genre, gen_tags]
  file = "recResults_df_items_adversarialAttack_with_embeddings_tag_global_7.csv"

elif scenario == files[10]:
  file = 'recResults_df_items_adversarialAttack_with_embeddings_generated_tag_local_1.csv'
elif scenario == files[11]:
  file = 'recResults_df_items_adversarialAttack_with_embeddings_generated_tag_global_1.csv'
elif scenario == files[12]:
  file = 'recResults_df_items_adversarialAttack_with_embeddings_generated_tag_local_3.csv'
elif scenario == files[13]:
  file = 'recResults_df_items_adversarialAttack_with_embeddings_generated_tag_global_3.csv'
elif scenario == files[14]:
  file = 'recResults_df_items_adversarialAttack_with_embeddings_generated_tag_local_5.csv'
elif scenario == files[15]:
  file = 'recResults_df_items_adversarialAttack_with_embeddings_generated_tag_global_5.csv'
elif scenario == files[16]:
  file = 'recResults_df_items_adversarialAttack_with_embeddings_generated_tag_local_7.csv'
elif scenario == files[17]:
  file = 'recResults_df_items_adversarialAttack_with_embeddings_generated_tag_global_7.csv'

#file4 = 'df_items_adversarialAttack_with_embeddings_tag_3.csv'
#file5 = 'df_items_adversarialAttack_with_embeddings_generated_tags_1.csv'
#file6 = 'df_items_adversarialAttack_with_embeddings_generated_tags_3.csv'
#file7 = 'df_items_adversarialAttack_with_embeddings_generated_tags_5.csv'
#file = 'df_items_adversarialAttack_with_embeddings_generated_tags_7.csv'

fileName = "drive_folder/" + file
candidResults = pd.read_csv(fileName)
display(candidResults)

candidResults.rename(columns={'rec_item_Rating': 'rec_items_Rating'}, inplace=True)

candidResults = candidResults[candidResults['userId'].isin(random_users)]
display(candidResults)



,userId,num_interactions,activity_frequency,mean_rating,mainstreamness,gt_items,rec_item_Rating,rec_items_Decay
0,1,164,0.000000,4.408537,0.907014,"[1219, 1348, 2644, 1258, 1278, 2459, 2648, 271...","[1562, 4232, 1894, 7743, 913, 198, 55854, 4889...","[1483, 55765, 136850, 60684, 3741, 288, 32587,..."
1,2,23,0.000000,3.847826,0.513185,"[80906, 89774, 1704, 122882, 114060, 80489]","[5220, 59369, 106916, 55765, 104879, 60684, 32...","[174055, 68554, 68358, 33794, 60684, 4161, 135..."
2,3,31,0.000000,2.935484,1.668526,"[6238, 1124, 688, 3949, 2090, 527, 5048, 2424]","[1214, 1240, 1320, 52281, 7743, 198, 2021, 442...","[1077, 1214, 1240, 4210, 1924, 7743, 2407, 198..."
3,4,146,3.308219,3.643836,1.038700,"[4033, 3967, 4027, 3408, 4014, 3897, 3317, 113...","[87413, 1244, 193, 47610, 1673, 1894, 49286, 9...","[7090, 1263, 3287, 6618, 3785, 59784, 708, 872..."
4,5,35,0.000000,3.600000,0.667279,"[410, 596, 266, 290, 475, 534, 247, 300, 474]","[1244, 193, 1673, 1894, 32587, 198, 55854, 488...","[87413, 1244, 5992, 6708, 259, 193, 4210, 198,..."
...,...,...,...,...,...,...,...,...
605,606,536,0.095149,3.746269,0.457491,"[7099, 3160, 3426, 1729, 8128, 920, 8622, 1246...","[87413, 193, 47610, 1673, 1894, 49286, 55854, ...","[313, 63239, 4467, 59103, 162578, 1894, 2116, ..."
606,607,140,0.135714,3.857143,0.847042,"[1249, 593, 1, 949, 1625, 3347, 919, 1023, 107...","[52281, 1275, 32587, 198, 55854, 4889, 170, 44...","[4901, 96829, 55290, 2443, 547, 55854, 509, 32..."
607,608,562,0.005338,3.122776,0.729380,"[2676, 5308, 5219, 4054, 93, 3316, 3355, 1772,...","[4232, 52281, 6550, 1894, 60684, 7743, 3784, 1...","[4232, 52281, 799, 6550, 2004, 3784, 198, 3452..."
608,609,29,0.000000,3.275862,0.391237,"[828, 1059, 1150, 650, 833, 892, 1056, 1161]","[1240, 32587, 198, 55854, 4889, 3259, 170, 442...","[4210, 198, 55854, 4889, 170, 225, 1422, 23, 2..."


,userId,num_interactions,activity_frequency,mean_rating,mainstreamness,gt_items,rec_items_Rating,rec_items_Decay
0,1,164,0.000000,4.408537,0.907014,"[1219, 1348, 2644, 1258, 1278, 2459, 2648, 271...","[1562, 4232, 1894, 7743, 913, 198, 55854, 4889...","[1483, 55765, 136850, 60684, 3741, 288, 32587,..."
2,3,31,0.000000,2.935484,1.668526,"[6238, 1124, 688, 3949, 2090, 527, 5048, 2424]","[1214, 1240, 1320, 52281, 7743, 198, 2021, 442...","[1077, 1214, 1240, 4210, 1924, 7743, 2407, 198..."
6,7,104,0.086538,3.514423,0.853002,"[8665, 4700, 8949, 4246, 8972, 30812, 2671, 31...","[4232, 5220, 68358, 60684, 52722, 198, 4889, 1...","[474, 996, 6708, 55765, 4210, 49530, 91978, 41..."
9,10,87,0.011494,3.339080,0.973910,"[103335, 95510, 72737, 70293, 33145, 8970, 951...","[1244, 4823, 89904, 5992, 47610, 6550, 1894, 9...","[1244, 4022, 5992, 104879, 1894, 2520, 4889, 3..."
10,11,51,0.215686,3.647059,0.655978,"[1721, 529, 1784, 318, 593, 1693, 1704, 36, 13...","[1240, 1275, 198, 4889, 315, 485, 5541, 23, 32...","[5220, 6548, 32587, 198, 4889, 315, 204, 2194,..."
...,...,...,...,...,...,...,...,...
603,604,80,0.000000,3.425000,0.560991,"[60, 112, 170, 327, 364, 14, 169, 454, 23, 610...","[367, 2389, 799, 3785, 1673, 32587, 198, 55854...","[367, 2389, 5220, 52281, 1673, 303, 60684, 325..."
605,606,536,0.095149,3.746269,0.457491,"[7099, 3160, 3426, 1729, 8128, 920, 8622, 1246...","[87413, 193, 47610, 1673, 1894, 49286, 55854, ...","[313, 63239, 4467, 59103, 162578, 1894, 2116, ..."
606,607,140,0.135714,3.857143,0.847042,"[1249, 593, 1, 949, 1625, 3347, 919, 1023, 107...","[52281, 1275, 32587, 198, 55854, 4889, 170, 44...","[4901, 96829, 55290, 2443, 547, 55854, 509, 32..."
608,609,29,0.000000,3.275862,0.391237,"[828, 1059, 1150, 650, 833, 892, 1056, 1161]","[1240, 32587, 198, 55854, 4889, 3259, 170, 442...","[4210, 198, 55854, 4889, 170, 225, 1422, 23, 2..."


# Re-ranking

In [ ]:
def get_recent_user_history(user_id, df_ratings, df_items, N_HISTORY=10):
    """
    Get the most recent N_HISTORY items that the user has interacted with.

    Parameters:
    - user_id: int, the ID of the user.
    - df_ratings: DataFrame, user ratings data with columns ['userId', 'itemId', 'rating', 'timestamp'].
    - df_items: DataFrame, item metadata with columns ['itemId', 'title', 'genres'].
    - N_HISTORY: int, number of recent user interactions to return.

    Returns:
    - DataFrame with columns ['title', 'genres'] of the most recent items.
    """
    # Get the user's ratings sorted by timestamp
    user_ratings = df_ratings[df_ratings['userId'] == user_id].sort_values(by='timestamp', ascending=False)
    # Keep only the most recent N_HISTORY items
    recent_ratings = user_ratings.head(N_HISTORY)
    # Merge with item metadata to get titles and genres
    recent_items = recent_ratings.merge(df_items, on='itemId', how='left')
    return recent_items[['title', 'genres']]

user_profile = get_recent_user_history(user_id=1, df_ratings=rating_df, df_items=df_items, N_HISTORY=10)
display(user_profile)


,title,genres
0,20 Dates (1998),Comedy|Romance
1,Back to the Future Part III (1990),Adventure|Comedy|Sci-Fi|Western
2,¡Three Amigos! (1986),Comedy|Western
3,Tombstone (1993),Action|Drama|Western
4,"Messenger: The Story of Joan of Arc, The (1999)",Drama|War
5,Pink Floyd: The Wall (1982),Drama|Musical
6,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller
7,"Good Morning, Vietnam (1987)",Comedy|Drama|War
8,Rob Roy (1995),Action|Drama|Romance|War
9,Henry V (1989),Action|Drama|Romance|War


In [ ]:
import re

def get_recent_user_history_detailed(user_id, df_ratings, df_items, N_HISTORY=10):
    """
    Get a detailed user profile including overall genre preferences, recent interactions, and movie years.

    Parameters:
    - user_id: int, the ID of the user.
    - df_ratings: DataFrame, user ratings data with columns ['userId', 'itemId', 'rating', 'timestamp'].
    - df_items: DataFrame, item metadata with columns ['itemId', 'title', 'genres'].
    - N_HISTORY: int, number of recent user interactions to consider.

    Returns:
    - user_profile: dict, with keys:
        - 'overall_preference': list of top 5 genres.
        - 'recent_interactions': DataFrame with columns ['title', 'genres', 'year'] of the most recent items.
    """
    # Get all interactions of the user
    user_ratings = df_ratings[df_ratings['userId'] == user_id].sort_values(by='timestamp', ascending=False)
    all_interactions = user_ratings.merge(df_items, on='itemId', how='left')

    # Extract year from 'title' if 'year' column doesn't exist
    if 'year' not in all_interactions.columns:
        # Define a function to extract year from title
        def extract_year(title):
            match = re.search(r'\((\d{4})\)', title)
            return int(match.group(1)) if match else None
        all_interactions['year'] = all_interactions['title'].apply(extract_year)

    # Calculate overall preference summary (e.g., most frequent genres)
    overall_preference = (
        all_interactions['genres']
        .str.split('|')
        .explode()
        .value_counts()
        .head(5)
        .index.tolist()
    )

    # Get most recent interactions
    recent_interactions = all_interactions.head(N_HISTORY)

    # Format the result
    user_profile = {
        "overall_preference": overall_preference,
        "recent_interactions": recent_interactions[['title', 'genres', 'year']]
    }

    return user_profile

user_profile = get_recent_user_history_detailed(user_id=1, df_ratings=rating_df, df_items=df_items, N_HISTORY=10)
display(user_profile)


{'overall_preference': ['Action', 'Comedy', 'Adventure', 'Drama', 'Thriller'],
 'recent_interactions':                                              title  \
 0                                  20 Dates (1998)   
 1               Back to the Future Part III (1990)   
 2                            ¡Three Amigos! (1986)   
 3                                 Tombstone (1993)   
 4  Messenger: The Story of Joan of Arc, The (1999)   
 5                      Pink Floyd: The Wall (1982)   
 6             Independence Day (a.k.a. ID4) (1996)   
 7                     Good Morning, Vietnam (1987)   
 8                                   Rob Roy (1995)   
 9                                   Henry V (1989)   
 
                              genres  year  
 0                    Comedy|Romance  1998  
 1   Adventure|Comedy|Sci-Fi|Western  1990  
 2                    Comedy|Western  1986  
 3              Action|Drama|Western  1993  
 4                         Drama|War  1999  
 5                   

In [ ]:
def get_candidate_items(user_id, rec_df, item_df, candid_colName='rec_items_Rating', N_CANDIDATES=20):
    """
    Get the candidate items for the specified user from the recommendation DataFrame, including movie years.

    Parameters:
    - user_id: int, the ID of the user.
    - rec_df: DataFrame, recommendations DataFrame with at least columns ['userId', candid_colName].
    - item_df: DataFrame, item metadata with columns ['itemId', 'title', 'genres'].
    - candid_colName: str, the name of the column in rec_df that contains the recommended item IDs.
    - N_CANDIDATES: int, number of candidate items to return.

    Returns:
    - DataFrame with columns ['userId', 'itemId', 'title', 'genres', 'year'] of candidate items.
    """
    # Get the list of candidate item IDs for the specified user
    rec_item_ids = rec_df.loc[rec_df['userId'] == user_id, candid_colName].values[0]

    # Ensure rec_item_ids is a list
    if isinstance(rec_item_ids, str):
        import ast
        rec_item_ids = ast.literal_eval(rec_item_ids)

    # Limit to top N_CANDIDATES items
    rec_item_ids = rec_item_ids[:N_CANDIDATES]

    # Filter item_df to get metadata
    item_df_filtered = item_df[item_df['itemId'].isin(rec_item_ids)].copy()
    item_df_filtered['itemId'] = pd.Categorical(item_df_filtered['itemId'], categories=rec_item_ids, ordered=True)
    item_df_filtered = item_df_filtered.sort_values('itemId')

    # Extract year from 'title' if 'year' column doesn't exist
    if 'year' not in item_df_filtered.columns:
        # Define a function to extract year from title
        def extract_year(title):
            match = re.search(r'\((\d{4})\)', title)
            return int(match.group(1)) if match else None
        item_df_filtered['year'] = item_df_filtered['title'].apply(extract_year)

    # Add the userId to the DataFrame
    item_df_filtered['userId'] = user_id

    return item_df_filtered[['userId', 'itemId', 'title', 'genres', 'year']]

candidate_items_df = get_candidate_items(user_id=56, rec_df=candidResults, item_df=df_items, candid_colName='rec_items_Rating', N_CANDIDATES=10)
display(candidate_items_df)


,userId,itemId,title,genres,year
546,56,648,Mission: Impossible (1996),Action|Adventure|Mystery|Thriller,1996
336,56,379,Timecop (1994),Action|Sci-Fi|Thriller,1994
1218,56,1617,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller,1997
5004,56,7743,Explorers (1985),Adventure|Children|Sci-Fi,1985
1399,56,1918,Lethal Weapon 4 (1998),Action|Comedy|Crime|Thriller,1998
5850,56,32587,Sin City (2005),Action|Crime|Film-Noir|Mystery|Thriller,2005
167,56,198,Strange Days (1995),Action|Crime|Drama|Mystery|Sci-Fi|Thriller,1995
3571,56,4889,Heist (2001),Crime|Drama,2001
274,56,315,"Specialist, The (1994)",Action|Drama|Thriller,1994
2446,56,3259,Far and Away (1992),Adventure|Drama|Romance,1992


In [ ]:
import re

def parse_standard_response(response_text, max_index):
    """
    Parse the LLM's response to extract the scores, handling various formats.

    Parameters:
    - response_text: str, the text response from the LLM.
    - max_index: int, the maximum index (number of items).

    Returns:
    - scores: list of tuples, each tuple is (index, score), where index is 0-based.
    """
    scores = []
    lines = response_text.strip().split('\n')
    for line in lines:
        # Use regex to find patterns like "1: score", "1. ... - score", etc.
        match = re.match(r'^\s*(\d+)[\.:]\s*(.*?)(?:-|:)?\s*(\d(?:\.\d+)?)\s*$', line)
        if match:
            idx = int(match.group(1)) - 1  # Adjust index to be 0-based
            score_str = match.group(3).strip()
            try:
                score = float(score_str)
                if 0 <= idx < max_index:
                    scores.append((idx, score))
            except ValueError:
                continue
    return scores


# Assume response_text is the text received from the LLM
response_text = """
1: 4.5
2: 3.0
3: 5.0
4: 2.5
5: 4.0
6: 3.5
7: 4.8
8: 3.2
9: 4.1
10: 2.9
"""

scores = parse_standard_response(response_text, max_index=10)
print(scores)


[(0, 4.5), (1, 3.0), (2, 5.0), (3, 2.5), (4, 4.0), (5, 3.5), (6, 4.8), (7, 3.2), (8, 4.1), (9, 2.9)]


In [ ]:
def create_standard_prompt(user_profile, candidate_items):
    """
    Create a prompt for the LLM based on the user's profile and candidate items,
    including overall years in general preference and removing years from movie lists.

    Parameters:
    - user_profile: dict, as returned by get_recent_user_history_detailed.
    - candidate_items: list of dicts, each dict represents a candidate item with keys ['userId', 'itemId', 'title', 'genres', 'year'].

    Returns:
    - prompt: str, the prompt to send to the LLM.
    """
    num_items = len(candidate_items)

    # Compute overall years from recent interactions
    years = user_profile['recent_interactions']['year'].dropna().astype(int)
    # Group years into decades and get the top 3 decades
    decades = (years // 10 * 10).value_counts().head(3).index.tolist()
    decades_str = ', '.join(f"{decade}s" for decade in sorted(decades))

    prompt = f"""Given a user who generally likes movies with genres: {', '.join(user_profile['overall_preference'])} and years: {decades_str}.

Recently, the user has liked the following movies:
"""
    for idx, row in user_profile['recent_interactions'].iterrows():
        prompt += f"- {row['title']} ({row['genres']})\n"

    prompt += f"\nPlease rate the likelihood that the user will enjoy each of the following {num_items} movies on a scale from 1 to 5 (1 = Not likely, 5 = Very likely):\n"

    for idx, item in enumerate(candidate_items):
        prompt += f"{idx + 1}. {item['title']} ({item['genres']})\n"

    prompt += f"\nPlease provide a rating for each of the {num_items} movies in the exact format:\n1: score\n2: score\n...\n{num_items}: score\nDo not include any additional text or commentary. Make sure to rate all {num_items} movies listed."
    #prompt += f"\nPlease provide a rating for each of the {num_items} movies in the format:\n1: score\n2: score\n...\n{num_items}: score\nMake sure to rate all {num_items} movies listed."
    return prompt.strip()

candidate_items_list = candidate_items_df.to_dict('records')
prompt = create_standard_prompt(user_profile, candidate_items_list)
print(prompt)


Given a user who generally likes movies with genres: Action, Comedy, Adventure, Drama, Thriller and years: 1980s, 1990s.

Recently, the user has liked the following movies:
- 20 Dates (1998) (Comedy|Romance)
- Back to the Future Part III (1990) (Adventure|Comedy|Sci-Fi|Western)
- ¡Three Amigos! (1986) (Comedy|Western)
- Tombstone (1993) (Action|Drama|Western)
- Messenger: The Story of Joan of Arc, The (1999) (Drama|War)
- Pink Floyd: The Wall (1982) (Drama|Musical)
- Independence Day (a.k.a. ID4) (1996) (Action|Adventure|Sci-Fi|Thriller)
- Good Morning, Vietnam (1987) (Comedy|Drama|War)
- Rob Roy (1995) (Action|Drama|Romance|War)
- Henry V (1989) (Action|Drama|Romance|War)

Please rate the likelihood that the user will enjoy each of the following 10 movies on a scale from 1 to 5 (1 = Not likely, 5 = Very likely):
1. Mission: Impossible (1996) (Action|Adventure|Mystery|Thriller)
2. Timecop (1994) (Action|Sci-Fi|Thriller)
3. L.A. Confidential (1997) (Crime|Film-Noir|Mystery|Thriller)
4. 

In [ ]:
import re

import re

def parse_response_with_explanations(response_text, max_index):
    """
    Parse the LLM's response to extract scores and explanations with a more structured format.

    Parameters:
    - response_text: str, the text response from the LLM.
    - max_index: int, the maximum index (number of items).

    Returns:
    - scores_and_explanations: list of tuples, each tuple is (index, score, explanation).
    """
    scores_and_explanations = []
    lines = response_text.strip().split('\n')

    for line in lines:
        # Adjusted regex to match the new format '1. Title (Genre) - Rating: score - Explanation'
        match = re.match(r'^\s*(\d+)[\.:]\s+.*?-\s+Rating:\s*(\d(?:\.\d+)?)\s*-\s*(.+)$', line)

        if match:
            idx = int(match.group(1)) - 1  # Adjust index to be 0-based
            score_str = match.group(2).strip()
            explanation = match.group(3).strip()

            try:
                score = float(score_str)
                if 0 <= idx < max_index:
                    scores_and_explanations.append((idx, score, explanation))
            except ValueError:
                continue

    return scores_and_explanations



response_text = """
1. Timecop (1994) - Rating: 4 - This movie fits well within the user's preferred genres of action and thriller. Sci-fi is less favored, but the era and genres overall match well.

2. Bad Boys (1995) - Rating: 5 - A perfect match for the user's interests in action, comedy, and crime, making it highly likely the user will enjoy this movie.

3. Species (1995) - Rating: 5 - The user has liked this movie before, so they are very likely to enjoy watching it again.

4. Jade (1995) - Rating: 3 - Although a thriller, it lacks the action and crime elements, making it less appealing to the user.

5. Surviving the Game (1994) - Rating: 4 - Aligns with action and thriller preferences, though it lacks comedy and crime.

6. Strange Days (1995) - Rating: 5 - Combines action, crime, drama, and thriller, matching the user’s taste and is from the 90s.

7. Specialist, The (1994) - Rating: 5 - The user has liked this movie previously, so it’s very likely they will enjoy it again.

8. Hackers (1995) - Rating: 4 - Matches action, crime, and thriller interests, but lacks comedy.

9. Junior (1994) - Rating: 3 - Contains comedy but lacks other preferred elements like action or thriller.

10. Last Action Hero (1993) - Rating: 5 - An action-comedy that aligns with the user’s 90s preferences.
"""

# Parse the response
scores_and_explanations = parse_response_with_explanations(response_text, max_index=10)
print(scores_and_explanations)



[(0, 4.0, "This movie fits well within the user's preferred genres of action and thriller. Sci-fi is less favored, but the era and genres overall match well."), (1, 5.0, "A perfect match for the user's interests in action, comedy, and crime, making it highly likely the user will enjoy this movie."), (2, 5.0, 'The user has liked this movie before, so they are very likely to enjoy watching it again.'), (3, 3.0, 'Although a thriller, it lacks the action and crime elements, making it less appealing to the user.'), (4, 4.0, 'Aligns with action and thriller preferences, though it lacks comedy and crime.'), (5, 5.0, 'Combines action, crime, drama, and thriller, matching the user’s taste and is from the 90s.'), (6, 5.0, 'The user has liked this movie previously, so it’s very likely they will enjoy it again.'), (7, 4.0, 'Matches action, crime, and thriller interests, but lacks comedy.'), (8, 3.0, 'Contains comedy but lacks other preferred elements like action or thriller.'), (9, 5.0, 'An action

In [ ]:
def create_standard_prompt_with_explanations(user_profile, candidate_items):
    """
    Create a prompt that asks the LLM to provide explanations for each rating in a structured format.

    Parameters:
    - user_profile: dict, as returned by get_recent_user_history_detailed.
    - candidate_items: list of dicts, each dict represents a candidate item with keys ['userId', 'itemId', 'title', 'genres', 'year'].

    Returns:
    - prompt: str, the prompt to send to the LLM.
    """
    num_items = len(candidate_items)

    # Compute overall years from recent interactions
    years = user_profile['recent_interactions']['year'].dropna().astype(int)
    # Group years into decades and get the top 3 decades
    decades = (years // 10 * 10).value_counts().head(3).index.tolist()
    decades_str = ', '.join(f"{decade}s" for decade in sorted(decades))

    prompt = f"""Given a user who generally likes movies with genres: {', '.join(user_profile['overall_preference'])} and years: {decades_str}.

Recently, the user has liked the following movies:
"""
    for idx, row in user_profile['recent_interactions'].iterrows():
        prompt += f"- {row['title']} ({row['genres']})\n"

    prompt += f"\nPlease rate the likelihood that the user will enjoy each of the following {num_items} movies on a scale from 1 to 5 (1 = Not likely, 5 = Very likely). For each rating, provide a brief explanation for your decision.\n"
    prompt += "Format each entry as:\n"
    prompt += "1. Title (Genre) - Rating: score - Explanation\n\n"

    for idx, item in enumerate(candidate_items):
        prompt += f"{idx + 1}. {item['title']} ({item['genres']})\n"

    prompt += f"\nPlease respond in the exact format provided, making sure to rate all {num_items} movies and provide an explanation for each."
    return prompt.strip()


candidate_items_list = candidate_items_df.to_dict('records')
prompt = create_standard_prompt_with_explanations(user_profile, candidate_items_list)
print(prompt)



Given a user who generally likes movies with genres: Action, Comedy, Adventure, Drama, Thriller and years: 1980s, 1990s.

Recently, the user has liked the following movies:
- 20 Dates (1998) (Comedy|Romance)
- Back to the Future Part III (1990) (Adventure|Comedy|Sci-Fi|Western)
- ¡Three Amigos! (1986) (Comedy|Western)
- Tombstone (1993) (Action|Drama|Western)
- Messenger: The Story of Joan of Arc, The (1999) (Drama|War)
- Pink Floyd: The Wall (1982) (Drama|Musical)
- Independence Day (a.k.a. ID4) (1996) (Action|Adventure|Sci-Fi|Thriller)
- Good Morning, Vietnam (1987) (Comedy|Drama|War)
- Rob Roy (1995) (Action|Drama|Romance|War)
- Henry V (1989) (Action|Drama|Romance|War)

Please rate the likelihood that the user will enjoy each of the following 10 movies on a scale from 1 to 5 (1 = Not likely, 5 = Very likely). For each rating, provide a brief explanation for your decision.
Format each entry as:
1. Title (Genre) - Rating: score - Explanation

1. Mission: Impossible (1996) (Action|Adv

In [ ]:
import openai

openai.api_key = api_key
def rerank_for_user(user_id, df_ratings, df_items, rec_df, candid_colName, N_HISTORY=10, N_CANDIDATES=20):
    """
    Re-rank candidate items for a user using LLM, including movie years.

    Parameters:
    - user_id: int, the ID of the user.
    - df_ratings: DataFrame, user ratings data with columns ['userId', 'itemId', 'rating', 'timestamp'].
    - df_items: DataFrame, item metadata with columns ['itemId', 'title', 'genres'].
    - rec_df: DataFrame, recommendations DataFrame with at least columns ['userId', candid_colName].
    - candid_colName: str, the name of the column in rec_df that contains the recommended item IDs.
    - N_HISTORY: int, number of recent user interactions to consider.
    - N_CANDIDATES: int, number of candidate items to re-rank.

    Returns:
    - re_ranked_item_ids: list, re-ranked list of item IDs.
    """
    # Step 1: Get user profile
    user_profile = get_recent_user_history_detailed(user_id, df_ratings, df_items, N_HISTORY)

    # Step 2: Get candidate items for the user
    candidate_items_df = get_candidate_items(user_id, rec_df, df_items, candid_colName, N_CANDIDATES)
    candidate_items = candidate_items_df.to_dict('records')

    max_index = len(candidate_items)

    # Step 3: Create prompt
    prompt = create_standard_prompt(user_profile, candidate_items)

    #print(prompt)
    # Step 4: Call the LLM to generate scores
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides scores for movie recommendations."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
        temperature=0
    )

    response_text = response.choices[0].message['content'].strip()

    #print(response_text)
    # Step 5: Parse response to get scores
    scores = parse_standard_response(response_text, max_index)
    #print(scores)

    # Step 6: Re-rank items based on scores
    scored_items = []
    for idx, score in scores:
        if idx < len(candidate_items):
            item = candidate_items[idx]
            item['score'] = score
            scored_items.append(item)

    re_ranked_items = sorted(scored_items, key=lambda x: x['score'], reverse=True)
    re_ranked_item_ids = [item['itemId'] for item in re_ranked_items]
    return re_ranked_item_ids


# Test for user 1
test_user_id = 56
candid_colName = 'rec_items_Rating'

re_ranked_item_ids = rerank_for_user(
    user_id=test_user_id,
    df_ratings=rating_df,
    df_items=df_items,
    rec_df=candidResults,
    candid_colName=candid_colName,
    N_HISTORY=15,
    N_CANDIDATES=25
)

print(f"Re-ranked item IDs for user {test_user_id} and column {candid_colName}:", re_ranked_item_ids)


Re-ranked item IDs for user 56 and column rec_items_Rating: [648, 32587, 198, 315, 442, 23, 733, 1036, 849, 379, 1617, 1918, 170, 2881, 377, 832, 533, 1034, 1479, 1616, 7743, 4889, 3259, 2587, 1625]


In [ ]:
def rerank_for_user_with_explanations(user_id, df_ratings, df_items, rec_df, candid_colName, N_HISTORY=10, N_CANDIDATES=20):
    """
    Re-rank candidate items for a user using LLM, including explanations.

    Parameters:
    - user_id: int, the ID of the user.
    - df_ratings: DataFrame of user ratings.
    - df_items: DataFrame of item metadata.
    - rec_df: DataFrame of recommendations.
    - candid_colName: str, column name for candidate items.
    - N_HISTORY: int, number of recent interactions.
    - N_CANDIDATES: int, number of candidate items.

    Returns:
    - re_ranked_item_ids: list of re-ranked item IDs.
    - explanations: list of explanations for each item.
    """
    # Step 1: Get user profile
    user_profile = get_recent_user_history_detailed(user_id, df_ratings, df_items, N_HISTORY)

    # Step 2: Get candidate items
    candidate_items_df = get_candidate_items(user_id, rec_df, df_items, candid_colName, N_CANDIDATES)
    candidate_items = candidate_items_df.to_dict('records')

    max_index = len(candidate_items)

    # Step 3: Create prompt
    prompt = create_standard_prompt_with_explanations(user_profile, candidate_items)

    #print(prompt)
    # Step 4: Call the LLM
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Use GPT-4 for better performance if available
        messages=[
            {"role": "system", "content": "You are an AI assistant that provides ratings and explanations for movie recommendations."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=2000,
        temperature=0
    )

    response_text = response.choices[0].message['content'].strip()

    #print(response_text)
    # Step 5: Parse response
    scores_and_explanations = parse_response_with_explanations(response_text, max_index)
    #print(scores_and_explanations)

    # Step 6: Re-rank items
    if not scores_and_explanations:
        print(f"No valid scores parsed for user {user_id}.")
        return [], []

    scored_items = []
    for idx, score, explanation in scores_and_explanations:
        if idx < len(candidate_items):
            item = candidate_items[idx]
            item['score'] = score
            item['explanation'] = explanation
            scored_items.append(item)

    re_ranked_items = sorted(scored_items, key=lambda x: x['score'], reverse=True)
    re_ranked_item_ids = [item['itemId'] for item in re_ranked_items]
    explanations = [item['explanation'] for item in re_ranked_items]

    return re_ranked_item_ids, explanations

# Test for user 1
test_user_id = 56
candid_colName = 'rec_items_Rating'

re_ranked_item_ids, explanations = rerank_for_user_with_explanations(
    user_id=test_user_id,
    df_ratings=rating_df,
    df_items=df_items,
    rec_df=candidResults,
    candid_colName=candid_colName,
    N_HISTORY=10,
    N_CANDIDATES=10
)

print("--------------------------------")
print(f"Re-ranked item IDs for user {test_user_id} and column {candid_colName}:", re_ranked_item_ids)
print("Explanations:", explanations)


--------------------------------
Re-ranked item IDs for user 56 and column rec_items_Rating: [648, 1918, 198, 315, 379, 1617, 32587, 4889, 3259, 7743]
Explanations: ["This movie aligns perfectly with the user's preference for action, thriller, and drama. It's also from the 1990s, which is the user's preferred decade for movies.", "This movie combines action, comedy, crime, and thriller, all of which are genres the user enjoys. It's also from the user's preferred decade.", "This movie aligns perfectly with the user's genre preferences and is from their preferred decade. The user might appreciate the mix of action, crime, drama, and sci-fi.", "This movie aligns perfectly with the user's genre preferences and is from their preferred decade. The user has also previously enjoyed this movie.", "This movie fits the user's interest in action and thriller genres. The addition of sci-fi might be appreciated given the user's enjoyment of 'Species' and 'Judge Dredd'.", "This movie combines crime a

# Block 1: Re-ranking Without Explanations

In [ ]:
import time

itt_count = 0

# Start the timer
start_time = time.time()

# List of recommendation columns to process
rec_cols = ['rec_items_Rating', 'rec_items_Decay']

# Initialize new columns in candidResults DataFrame for re-ranked items without explanations
for col in rec_cols:
    new_col_reranked = f"{col}_reranked"
    candidResults[new_col_reranked] = None
    candidResults[new_col_reranked] = candidResults[new_col_reranked].astype(object)

# Iterate over each user to process re-ranking without explanations
user_ids = candidResults['userId'].unique()

for user_id in user_ids:
    time.sleep(0.3)  # Pause before retrying
    itt_count += 1  # Increment the counter
    # Print the user ID every 50 iterations
    if itt_count % 50 == 1:
        print(f"Processing user {user_id} at iteration {itt_count}")

    for col in rec_cols:
        new_col_reranked = f"{col}_reranked"
        re_ranked_item_ids = []
        retries = 0
        max_retries = 3

        # Retry mechanism for re-ranking without explanations
        while retries < max_retries and not re_ranked_item_ids:
            try:
                re_ranked_item_ids = rerank_for_user(
                    user_id=user_id,
                    df_ratings=rating_df,
                    df_items=df_items,
                    rec_df=candidResults,
                    candid_colName=col,
                    N_HISTORY=10,
                    N_CANDIDATES=25
                )
            except Exception as e:
                print(f"Error for user {user_id} and column {col} without explanations: {e}")
                retries += 1
                time.sleep(1)  # Pause before retrying

        # Update the DataFrame with the re-ranked items without explanations
        index = candidResults.index[candidResults['userId'] == user_id][0]
        candidResults.at[index, new_col_reranked] = re_ranked_item_ids

# End the timer
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Computational time for re-ranking: {elapsed_time:.2f} seconds")

Processing user 1 at iteration 1
Processing user 136 at iteration 51
Processing user 281 at iteration 101
Processing user 454 at iteration 151
Computational time for re-ranking: 802.63 seconds


In [ ]:
display(candidResults)

,userId,num_interactions,activity_frequency,mean_rating,mainstreamness,gt_items,rec_items_Rating,rec_items_Decay,rec_items_Rating_reranked,rec_items_Decay_reranked
0,1,164,0.000000,4.408537,0.907014,"[1219, 1348, 2644, 1258, 1278, 2459, 2648, 271...","[1562, 4232, 1894, 7743, 913, 198, 55854, 4889...","[1483, 55765, 136850, 60684, 3741, 288, 32587,...","[4232, 198, 2968, 170, 485, 23, 1262, 1129, 84...","[1036, 55765, 60684, 288, 32587, 198, 2194, 85..."
2,3,31,0.000000,2.935484,1.668526,"[6238, 1124, 688, 3949, 2090, 527, 5048, 2424]","[1214, 1240, 1320, 52281, 7743, 198, 2021, 442...","[1077, 1214, 1240, 4210, 1924, 7743, 2407, 198...","[1240, 541, 1214, 7743, 2021, 442, 3959, 1127,...","[1240, 541, 3527, 1200, 1214, 7743, 198, 442, ..."
6,7,104,0.086538,3.514423,0.853002,"[8665, 4700, 8949, 4246, 8972, 30812, 2671, 31...","[4232, 5220, 68358, 60684, 52722, 198, 4889, 1...","[474, 996, 6708, 55765, 4210, 49530, 91978, 41...","[68358, 2571, 4232, 60684, 52722, 4887, 733, 5...","[55765, 49530, 36529, 996, 6708, 62374, 4326, ..."
9,10,87,0.011494,3.339080,0.973910,"[103335, 95510, 72737, 70293, 33145, 8970, 951...","[1244, 4823, 89904, 5992, 47610, 6550, 1894, 9...","[1244, 4022, 5992, 104879, 1894, 2520, 4889, 3...","[89904, 54259, 56152, 1244, 4823, 47610, 1894,...","[1244, 5992, 417, 5902, 6711, 63082, 1721, 402..."
10,11,51,0.215686,3.647059,0.655978,"[1721, 529, 1784, 318, 593, 1693, 1704, 36, 13...","[1240, 1275, 198, 4889, 315, 485, 5541, 23, 32...","[5220, 6548, 32587, 198, 4889, 315, 204, 2194,...","[1240, 198, 315, 23, 3256, 464, 51935, 1036, 5...","[32587, 198, 4889, 2194, 23, 464, 3703, 37733,..."
...,...,...,...,...,...,...,...,...,...,...
603,604,80,0.000000,3.425000,0.560991,"[60, 112, 170, 327, 364, 14, 169, 454, 23, 610...","[367, 2389, 799, 3785, 1673, 32587, 198, 55854...","[367, 2389, 5220, 52281, 1673, 303, 60684, 325...","[367, 177, 2719, 2389, 799, 3785, 32587, 198, ...","[367, 2389, 5220, 1673, 303, 60684, 32587, 198..."
605,606,536,0.095149,3.746269,0.457491,"[7099, 3160, 3426, 1729, 8128, 920, 8622, 1246...","[87413, 193, 47610, 1673, 1894, 49286, 55854, ...","[313, 63239, 4467, 59103, 162578, 1894, 2116, ...","[47610, 1894, 49286, 417, 866, 46976, 5902, 39...","[162578, 1, 63239, 59103, 1894, 54259, 56152, ..."
606,607,140,0.135714,3.857143,0.847042,"[1249, 593, 1, 949, 1625, 3347, 919, 1023, 107...","[52281, 1275, 32587, 198, 55854, 4889, 170, 44...","[4901, 96829, 55290, 2443, 547, 55854, 509, 32...","[32587, 1275, 198, 170, 485, 541, 733, 37733, ...","[4901, 96829, 55290, 547, 3256, 4450, 2908, 18..."
608,609,29,0.000000,3.275862,0.391237,"[828, 1059, 1150, 650, 833, 892, 1056, 1161]","[1240, 32587, 198, 55854, 4889, 3259, 170, 442...","[4210, 198, 55854, 4889, 170, 225, 1422, 23, 2...","[32587, 1240, 198, 3259, 170, 442, 23, 2881, 7...","[198, 2881, 4210, 4889, 170, 225, 1422, 23, 73..."


In [ ]:
import pandas as pd
import ast

def convert_stringified_lists(df, columns):
    """
    Convert string representations of lists in specified columns to actual lists of integers.

    Parameters:
    - df: DataFrame, the DataFrame to process.
    - columns: list, columns in which to check and convert string representations of lists.

    Returns:
    - None, modifies the DataFrame in place.
    """
    for col in columns:
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)

# Example usage:
columns_to_convert = ['gt_items', 'rec_items_Rating', 'rec_items_Decay', 'rec_items_Rating_reranked', 'rec_items_Decay_reranked']
convert_stringified_lists(candidResults, columns_to_convert)

# Verify the conversion
print(candidResults[columns_to_convert].head())


                                             gt_items  \
0   [1219, 1348, 2644, 1258, 1278, 2459, 2648, 271...   
2      [6238, 1124, 688, 3949, 2090, 527, 5048, 2424]   
6   [8665, 4700, 8949, 4246, 8972, 30812, 2671, 31...   
9   [103335, 95510, 72737, 70293, 33145, 8970, 951...   
10  [1721, 529, 1784, 318, 593, 1693, 1704, 36, 13...   

                                     rec_items_Rating  \
0   [1562, 4232, 1894, 7743, 913, 198, 55854, 4889...   
2   [1214, 1240, 1320, 52281, 7743, 198, 2021, 442...   
6   [4232, 5220, 68358, 60684, 52722, 198, 4889, 1...   
9   [1244, 4823, 89904, 5992, 47610, 6550, 1894, 9...   
10  [1240, 1275, 198, 4889, 315, 485, 5541, 23, 32...   

                                      rec_items_Decay  \
0   [1483, 55765, 136850, 60684, 3741, 288, 32587,...   
2   [1077, 1214, 1240, 4210, 1924, 7743, 2407, 198...   
6   [474, 996, 6708, 55765, 4210, 49530, 91978, 41...   
9   [1244, 4022, 5992, 104879, 1894, 2520, 4889, 3...   
10  [5220, 6548, 32587, 198, 

In [ ]:
print(len(candidResults.iloc[0]['rec_items_Rating_reranked']))
print(len(candidResults.iloc[0]['rec_items_Decay_reranked']))


25
25


In [ ]:
candidResults.iloc[1]['rec_items_Rating']
candidResults.to_csv('candidResults_reranked_' +file, index=False)


In [ ]:
# Modify the code to handle potential NaN or non-list entries in recommendation columns
def safe_recall_at_k(gt_items, rec_items, k=10):
    if not isinstance(rec_items, list):
        return 0  # return 0 if rec_items is not a list
    hits = len(set(gt_items).intersection(set(rec_items[:k])))
    return hits / min(len(gt_items), k)

def safe_mrr_at_k(gt_items, rec_items, k=10):
    if not isinstance(rec_items, list):
        return 0
    for rank, item in enumerate(rec_items[:k], start=1):
        if item in gt_items:
            return 1 / rank
    return 0

def safe_average_precision_at_k(gt_items, rec_items, k=10):
    if not isinstance(rec_items, list):
        return 0
    hits, precision_sum = 0, 0
    for rank, item in enumerate(rec_items[:k], start=1):
        if item in gt_items:
            hits += 1
            precision_sum += hits / rank
    return precision_sum / min(len(gt_items), k) if gt_items else 0

def safe_ndcg_at_k(gt_items, rec_items, k=10):
    if not isinstance(rec_items, list):
        return 0
    dcg, idcg = 0, 0
    for rank, item in enumerate(rec_items[:k], start=1):
        if item in gt_items:
            dcg += 1 / np.log2(rank + 1)
    for rank in range(1, min(len(gt_items), k) + 1):
        idcg += 1 / np.log2(rank + 1)
    return dcg / idcg if idcg > 0 else 0

# Define the metrics list explicitly
metrics = ["recall", "mrr", "map", "ndcg"]

# Compute metrics only for the last four columns
rec_columns = ["rec_items_Rating", "rec_items_Decay", "rec_items_Rating_reranked", "rec_items_Decay_reranked"]
safe_results = {metric: {col: [] for col in rec_columns} for metric in metrics}

for index, row in candidResults.iterrows():
    gt_items = row["gt_items"]
    for col in rec_columns:
        rec_items = row[col]
        safe_results["recall"][col].append(safe_recall_at_k(gt_items, rec_items))
        safe_results["mrr"][col].append(safe_mrr_at_k(gt_items, rec_items))
        safe_results["map"][col].append(safe_average_precision_at_k(gt_items, rec_items))
        safe_results["ndcg"][col].append(safe_ndcg_at_k(gt_items, rec_items))

# Convert individual user metric values into a DataFrame
safe_recall_df = pd.DataFrame(safe_results["recall"]).add_prefix("Recall_")
safe_mrr_df = pd.DataFrame(safe_results["mrr"]).add_prefix("MRR_")
safe_map_df = pd.DataFrame(safe_results["map"]).add_prefix("MAP_")
safe_ndcg_df = pd.DataFrame(safe_results["ndcg"]).add_prefix("NDCG_")

# Concatenate individual metrics for each user
user_metrics_df = pd.concat([safe_recall_df, safe_mrr_df, safe_map_df, safe_ndcg_df], axis=1)
user_metrics_df.insert(0, "userId", candidResults["userId"])

# Calculate average values for each metric across all users
average_metrics = user_metrics_df.mean(numeric_only=True).to_frame().T
average_metrics.index = ["Average"]

# Display results
final_output_df = pd.concat([user_metrics_df, average_metrics], ignore_index=False)
display(final_output_df)

,userId,Recall_rec_items_Rating,Recall_rec_items_Decay,Recall_rec_items_Rating_reranked,Recall_rec_items_Decay_reranked,MRR_rec_items_Rating,MRR_rec_items_Decay,MRR_rec_items_Rating_reranked,MRR_rec_items_Decay_reranked,MAP_rec_items_Rating,MAP_rec_items_Decay,MAP_rec_items_Rating_reranked,MAP_rec_items_Decay_reranked,NDCG_rec_items_Rating,NDCG_rec_items_Decay,NDCG_rec_items_Rating_reranked,NDCG_rec_items_Decay_reranked
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,197.000000,0.000000,0.100000,0.000000,0.100000,0.000000,0.500000,0.00000,0.333333,0.000000,0.050000,0.000000,0.033333,0.000000,0.138862,0.000000,0.110046
197,NaN,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.00000,0.142857,0.000000,0.000000,0.000000,0.014286,0.000000,0.000000,0.000000,0.073364
198,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
199,200.000000,0.300000,0.300000,0.500000,0.300000,0.500000,1.000000,1.00000,0.200000,0.150000,0.170000,0.344444,0.075000,0.315163,0.368856,0.567227,0.218195


#Block 2: Re-ranking With Explanations

In [ ]:
import time
# Initialize new columns in candidResults DataFrame for re-ranked items with explanations
for col in rec_cols:
    new_col_reranked_expl = f"{col}_reranked_expl"
    candidResults[new_col_reranked_expl] = None
    candidResults[new_col_reranked_expl] = candidResults[new_col_reranked_expl].astype(object)

# Iterate over each user to process re-ranking with explanations
for user_id in user_ids:
    for col in rec_cols:
        new_col_reranked_expl = f"{col}_reranked_expl"
        re_ranked_item_ids_expl = []
        explanations = []
        retries = 0
        max_retries = 3

        # Retry mechanism for re-ranking with explanations
        while retries < max_retries and not re_ranked_item_ids_expl:
            try:
                re_ranked_item_ids_expl, explanations = rerank_for_user_with_explanations(
                    user_id=user_id,
                    df_ratings=rating_df,
                    df_items=df_items,
                    rec_df=candidResults,
                    candid_colName=col,
                    N_HISTORY=10,
                    N_CANDIDATES=10
                )
            except Exception as e:
                print(f"Error for user {user_id} and column {col} with explanations: {e}")
                retries += 1
                time.sleep(1)  # Pause before retrying

        # Update the DataFrame with the re-ranked items and explanations
        index = candidResults.index[candidResults['userId'] == user_id][0]
        candidResults.at[index, new_col_reranked_expl] = list(zip(re_ranked_item_ids_expl, explanations))


In [ ]:
display(candidResults)

In [ ]:
candidResults.to_csv('candidResults_reranked.csv', index=False)

In [ ]:
import pandas as pd
import ast

# Assuming the dataframes are already defined: candidResult_org, candidResult_Attacked_tag1, and df_items_with_populairty

# -------------------------------------------------------
# Step 1: Ensure that recItemId is a list of integers
def ensure_list_of_ints(item_list):
    if isinstance(item_list, str):  # If item_list is a string, convert it to a list
        item_list = ast.literal_eval(item_list)  # Converts string representation of list to actual list
    return [int(item) for item in item_list]  # Ensure all items are integers

# Apply the conversion to both DataFrames
candidResults['rec_items_Rating'] = candidResults['rec_items_Rating'].apply(ensure_list_of_ints)
candidResults['rec_items_Decay'] = candidResults['rec_items_Decay'].apply(ensure_list_of_ints)

# -------------------------------------------------------
# Step 2: Create a dictionary mapping itemId to popularity_level
item_popularity_dict = df_items_with_populairty.set_index('itemId')['popularity_level'].to_dict()

# Helper functions
def get_popularity_levels(item_ids):
    return [item_popularity_dict.get(item_id, None) for item_id in item_ids]

def compute_average_popularity(popularity_levels):
    valid_levels = [level for level in popularity_levels if level is not None]
    return sum(valid_levels) / len(valid_levels) if valid_levels else None

# Define thresholds
LONG_TAIL_LEVEL = 0
MID_TAIL_LEVEL = 1
POPULAR_LEVEL = 2

def count_long_tail_items(popularity_levels):
    return sum(1 for level in popularity_levels if level == LONG_TAIL_LEVEL)

def count_mid_tail_items(popularity_levels):
    return sum(1 for level in popularity_levels if level == MID_TAIL_LEVEL)

def count_popular_items(popularity_levels):
    return sum(1 for level in popularity_levels if level == POPULAR_LEVEL)

# -------------------------------------------------------
# Apply functions to both dataframes (candidResult_org and candidResult_Attacked_tag1)
for df in [candidResults]:
    # Apply functions to compute popularity levels for each recItemId list
    df['recItemId_popularity_levels_Rating'] = df['rec_item_Rating'].apply(get_popularity_levels)
    df['avg_recItemId_popularity_Rating'] = df['recItemId_popularity_levels_Rating'].apply(compute_average_popularity)
    df['num_long_tail_items_Rating'] = df['recItemId_popularity_levels_Rating'].apply(count_long_tail_items)
    df['num_mid_tail_items_Rating'] = df['recItemId_popularity_levels_Rating'].apply(count_mid_tail_items)
    df['num_popular_items_Rating'] = df['recItemId_popularity_levels_Rating'].apply(count_popular_items)

# Apply functions to both dataframes (candidResult_org and candidResult_Attacked_tag1)
for df in [candidResults]:
    # Apply functions to compute popularity levels for each recItemId list
    df['recItemId_popularity_levels_Decay'] = df['rec_items_Decay'].apply(get_popularity_levels)
    df['avg_recItemId_popularity_Decay'] = df['recItemId_popularity_levels_Decay'].apply(compute_average_popularity)
    df['num_long_tail_items_Decay'] = df['recItemId_popularity_levels_Decay'].apply(count_long_tail_items)
    df['num_mid_tail_items_Decay'] = df['recItemId_popularity_levels_Decay'].apply(count_mid_tail_items)
    df['num_popular_items_Decay'] = df['recItemId_popularity_levels_Decay'].apply(count_popular_items)



# -------------------------------------------------------
# Reporting averages for columns in both DataFrames
print("Average metrics for candidResult_org (Avg. Rating):")
print(f"Average length of recItemId lists (Avg. Rating): {candidResults['recItemId_popularity_levels_Rating'].apply(len).mean()}")
print(f"Average popularity level of recommended items (Avg. Rating): {candidResults['avg_recItemId_popularity_Rating'].mean()}")
print(f"Average number of long-tail items per list (Avg. Rating): {candidResults['num_long_tail_items_Rating'].mean()}")
print(f"Average number of mid-tail items per list (Avg. Rating): {candidResults['num_mid_tail_items_Rating'].mean()}")
print(f"Average number of popular items per list (Avg. Rating): {candidResults['num_popular_items_Rating'].mean()}")

print("----------------------------------------")
print("----------------------------------------")
# -------------------------------------------------------
# Reporting averages for columns in both DataFrames
print("Average metrics for candidResult_org (Decay):")
print(f"Average length of recItemId lists (Decay): {candidResults['recItemId_popularity_levels_Decay'].apply(len).mean()}")
print(f"Average popularity level of recommended items (Decay): {candidResults['avg_recItemId_popularity_Decay'].mean()}")
print(f"Average number of long-tail items per list (Decay): {candidResults['num_long_tail_items_Decay'].mean()}")
print(f"Average number of mid-tail items per list (Decay): {candidResults['num_mid_tail_items_Decay'].mean()}")
print(f"Average number of popular items per list (Decay): {candidResults['num_popular_items_Decay'].mean()}")


Average metrics for candidResult_org (Avg. Rating):
Average length of recItemId lists (Avg. Rating): 25.0
Average popularity level of recommended items (Avg. Rating): 1.1106543983476476
Average number of long-tail items per list (Avg. Rating): 2.945
Average number of mid-tail items per list (Avg. Rating): 14.635
Average number of popular items per list (Avg. Rating): 5.635
----------------------------------------
----------------------------------------
Average metrics for candidResult_org (Decay):
Average length of recItemId lists (Decay): 25.0
Average popularity level of recommended items (Decay): 1.1346020594965676
Average number of long-tail items per list (Decay): 2.505
Average number of mid-tail items per list (Decay): 14.94
Average number of popular items per list (Decay): 5.77
